In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_all_links(url, depth=0, visited={}):
  headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
  response = requests.get(url,headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  links = soup.find_all('a')
  #links = [link.get('href') for link in links]
  links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#') and not link.get('href').startswith(url)]
  links = [urljoin(url, link) for link in links if link.startswith('/')]
  #links = [urljoin(url, link) for link in links if link]
  #print(links)


  # Recursively crawl the links at the next depth level
  if depth < 1: #3
    new_links = []
    for link in links:
      # Increment the visit count for the link
      if link in visited:
        visited[link] += 1
      else:
        visited[link] = 1
      # Get the newly-crawled links and add them to the list
        new_links.extend(get_all_links(link, depth=depth+1, visited=visited))
      # Add the newly-crawled links to the original list
    links.extend(new_links)

  return visited

base_url = 'https://www.thairath.co.th/home'
website_dict = get_all_links(base_url, depth=0, visited={})
print(website_dict)



{'https://www.thairath.co.th/news/politic/2599601': 4, 'https://www.thairath.co.th/news/politic/2599590': 4, 'https://www.thairath.co.th/news/local/bangkok/2599595': 4, 'https://www.thairath.co.th/business/economics/2598810': 4, 'https://www.thairath.co.th/news/local/2599005': 4, 'https://www.thairath.co.th/scoop/interview/2599235': 4, 'https://www.thairath.co.th/spotlight': 2, 'https://www.thairath.co.th/column': 2, 'https://www.thairath.co.th/news/local/bangkok/2598825': 2, 'https://www.thairath.co.th/column/newspaper/hehapatee': 1, 'https://www.thairath.co.th/news/politic/2599566': 1, 'https://www.thairath.co.th/column/newspaper/wikroh': 1, 'https://www.thairath.co.th/news/politic/2598873': 1, 'https://www.thairath.co.th/column/newspaper/kaablook': 1, 'https://www.thairath.co.th/news/local/2598900': 1, 'https://www.thairath.co.th/column/newspaper/thai_remark': 1, 'https://www.thairath.co.th/news/politic/2598765': 1, 'https://www.thairath.co.th/column/newspaper/greenhead': 1, 'https:

In [62]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = word_tokenize(self.sentence, engine="newmm")
        self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
        for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
            if key.isalpha():
            # If the key is text, add it to the new dictionary
                self.keyword_result[key] = self.keyword_dict[key]
        return self.keyword_result
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result


In [66]:
a = ['อาคาร 12 ชั้น 5 เลขที่ 1 ถนนวิภาวดีรังสิต แขวงจอมพล เขตจตุจักร กรุงเทพฯ 10900']
a = Thai(a)
a.sentence


'อาคาร 12 ชั้น 5 เลขที่ 1 ถนนวิภาวดีรังสิต แขวงจอมพล เขตจตุจักร กรุงเทพฯ 10900'

In [3]:
import unicodedata
import requests
from bs4 import BeautifulSoup

def scrape_tags(url):
  # Make a request to the website
  response = requests.get(url)
  # Parse the HTML of the website
  soup = BeautifulSoup(response.text, 'html.parser')
  # Find all p tags and title tags
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  #p_list = Thai(p_list) 
  #p_list=p_list.summarize
  
  
  
  return p_list, title_tag


for website in website_dict.keys():
  p, title = scrape_tags(website)
  print(f'For website {website}\n the p tags is: {p} \n the title tag is: {title}')



  
  


For website https://www.thairath.co.th/news/politic/2599601
 the p tags is: ['“ประยุทธ์-ประวิตร” เก็บอาการไม่ตอบสื่อ นายกฯปิดปากไม่พูดปมสงครามตัวแทน 2 ป. กําชับ ครม. ร่วมแรงตอบซักฟอก ม.152 เที่ยวสุดท้าย “พี่ป้อม” บอกไม่คุยนายกฯก่อนลงสนามเลือกตั้ง ยันยังคบกันเหมือนเดิม ยิ้มรับลงสนามชิงเก้าอี้กับ “น้องตู่” พปชร.จัดทัพฮึดสู้ชนทุกค่าย 30 ม.ค. จัดดินเนอร์ทอล์กระดมทุน ขายโต๊ะละ 3 ล้าน หน.พปชร.ขึ้นเวทีโชว์วิชั่น พร้อมทีมขับเคลื่อนเศรษฐกิจ สั่งสแกนยิบคนบริจาคเงินเข้าพรรค “สันติ” ลั่นไม่มีดีลพรรคพี่พรรคน้องหลีกทาง รทสช. “วิรัช” บอกเลี่ยงไม่ได้สองพี่น้องฟัดกันเอง ลูกหาบระส่ําหนัก ปากน้ําโพแพแตก 3 ส.ส.ถูกดูดหายเกลี้ยง “วีระกร” ไขก๊อกหนีตายซบ ภท. “นิโรธ” บ่นอึดอัด แต่แทงกั๊กย้ายเข้า รทสช. “สมชัย” แซะอีเวนต์เปิดตัว “บิ๊กตู่” ส่อผิดกฎหมายเลือกตั้งอื้อ เข้าข่ายขนคน-จัดเลี้ยง-มีแสดงมหรสพ จับพิรุธประชุมทิพย์ ขู่ กกต.เกียร์ว่างระวังงานเข้า', 'จากการที่ พล.อ.ประยุทธ์ จันทร์โอชา นายกรัฐมนตรีและ รมว.กลาโหม เปิดตัวสมัครเข้าเป็นสมาชิกพรรครวมไทยสร้างชาติ (รทสช.) ไปเมื่อวันที่ 9 ม.ค. ส่งผลกระทบไปถึงพรรคพลังประ